In [1]:
# import step
import pandas as pd
import json

In [7]:
keys = {}
with open('keys.json', 'r') as file:
    keys = json.load(file)

In [4]:
# set file path (up to your device)
file_path_name = keys['destScientificNameFile']
file_path_relation = keys['destRelationFile']
print(file_path_name)
print(file_path_relation)

./data/scientific_name.csv
./data/relation.csv


In [5]:
# read data from path above
names_df = pd.read_csv(file_path_name)
relations_df = pd.read_csv(file_path_relation)

In [6]:
names_df

,id,name
0,0,0
1,1,root
2,2,Bacteria
3,3,0
4,4,0
...,...,...
3070950,3070950,0
3070951,3070951,0
3070952,3070952,0
3070953,3070953,0


In [5]:
relations_df

,id,parent_id,rank
0,0,0,0
1,1,0,0
2,2,131567,superkingdom
3,3,0,0
4,4,0,0
...,...,...,...
3070950,3070950,0,0
3070951,3070951,0,0
3070952,3070952,0,0
3070953,3070953,0,0


In [25]:
allDf = relations_df
allDf['name'] = names_df['name']
print(allDf)


              id  parent_id          rank           name
0              0          0             0              0
1              1          0             0           root
2              2     131567  superkingdom       Bacteria
3              3          0             0              0
4              4          0             0              0
...          ...        ...           ...            ...
3070950  3070950          0             0              0
3070951  3070951          0             0              0
3070952  3070952          0             0              0
3070953  3070953          0             0              0
3070954  3070954      82115         genus  Ectorhizobium

[3070955 rows x 4 columns]


In [6]:
taxonomy_level7 = relations_df[relations_df['rank'] == "species"]
taxonomy_level2 = relations_df[relations_df['rank'] == "1"]
taxonomy_all = relations_df[relations_df['rank'] != "0"]

In [7]:
taxonomy_all


,id,parent_id,rank
2,2,131567,superkingdom
6,6,335928,genus
7,7,6,species
9,9,32199,species
10,10,1706371,genus
...,...,...,...
3070616,3070616,3059992,no rank
3070820,3070820,3060827,no rank
3070859,3070859,4783,species
3070949,3070949,82115,genus


In [8]:
relations_df[relations_df['rank'] == 'phylum']

,id,parent_id,rank
976,976,68336,phylum
1090,1090,68336,phylum
1117,1117,1798711,phylum
1224,1224,2,phylum
1239,1239,1783272,phylum
...,...,...,...
3036961,3036961,1935183,phylum
3036962,3036962,1935183,phylum
3039322,3039322,2,phylum
3039652,3039652,2,phylum


In [9]:
relations_df['rank'].unique()

array(['0', 'superkingdom', 'genus', 'species', 'order', 'family',
       'subspecies', 'no rank', 'subfamily', 'strain', 'serogroup',
       'biotype', 'tribe', 'phylum', 'class', 'species group', 'forma',
       'clade', 'suborder', 'subclass', 'varietas', 'kingdom',
       'subphylum', 'forma specialis', 'isolate', 'superfamily',
       'infraorder', 'infraclass', 'superorder', 'subgenus', 'superclass',
       'parvorder', 'serotype', 'species subgroup', 'subcohort', 'cohort',
       'genotype', 'subtribe', 'section', 'series', 'morph', 'subkingdom',
       'superphylum', 'subsection', 'pathogroup'], dtype=object)

In [28]:
def getTaxonomyName(id):
    try:
        id = int(id)
        name = names_df.loc[id]['name']
        return name
    except:
        print(f'error name of {id} is not define')
        return "none"

def getParentByTaxonomyId(id):
    try:
        id = int(id)
        parent = relations_df.loc[id]['parent_id']
        return parent
    except:
        print(f'there is some error while find parent of {id} in relataions_df !')
        return 1

def getRankByTaxonomy(id):
    try:
        id = int(id)
        rank = relations_df.loc[id]['rank']
        return rank
    except:
        print(f'there is some error while find rank of {id} in relataions_df !')
        return -1

def printProgress(r, s):
    progress = (r/s)*100
    print(f"Progress: {progress:.2f}% complete", end="\r")

def getNameParentRankById(id):
    try:
        id = int(id)
        data = allDf.loc[id]
        name = data['name']
        parent = data['parent_id']
        rank = data['rank']
        return name, int(parent), rank
    except:
        print(f'there is some error while find parent and rank of {id} in relataions_df !')
        return 1, '0'

In [11]:
getRankByTaxonomy(419352)
# relations_df.loc[419352]['parent_id']

'species'

In [12]:
def proofRootIs1(name, realation):
    sampled_df = taxonomy_all.sample(frac=0.0001)
    sampleSize = len(sampled_df)
    # sample = [id, name]
    for i, sample in enumerate(sampled_df.values):
        id = sample[0]
        try:
            while int(id) != 1:
                id = getParentByTaxonomyId(id)
        except:
            print(f'case id = {sample[0]} is no have parent that rank is 1')
        printProgress(i, sampleSize)


In [13]:
proofRootIs1(names_df, relations_df)

In [14]:
def createTaxonomy(d):
    return dict({'id': d[0], 'parent': d[1], 'rank': d[2]})

def createTaxonomyInfo(d):
    return dict({'data': createTaxonomy(d), 'children': []})



In [31]:
def createTaxonomyTree(sample):
    # list of TaxonomyInfo
    taxonomyList = []

    sampleSize = len(sample)
    
    # data = [id, parent, rank]
    for i, data in enumerate(sample.values):
        # taxonomyData = createTaxonomy(data)
        id = int(data[0])
        
        taxonomyPath = [getTaxonomyName(id)]
        taxonomyRankPath = [getRankByTaxonomy(id)]
        while id != 1:
            id = int(getParentByTaxonomyId(id))

            name, parent, rank = getNameParentRankById(id)
            taxonomyPath.append(name)
            taxonomyRankPath.append(rank)
        
        taxonomyList.append([int(data[0]), taxonomyPath[::-1],taxonomyRankPath[::-1]])

        # print progress
        printProgress(i, sampleSize)
    return taxonomyList

In [32]:
# sample = taxonomy_level7.sample(frac=0.001)
sample = taxonomy_all.sample(frac=0.01)
taxonomyInfoRoot = createTaxonomyTree(sample)


In [33]:
taxonomyInfoRoot

[[2889794,
  ['cellular organisms',
   'Eukaryota',
   'Opisthokonta',
   'Metazoa',
   'Eumetazoa',
   'Bilateria',
   'Protostomia',
   'Ecdysozoa',
   'Panarthropoda',
   'Arthropoda',
   'Mandibulata',
   'Pancrustacea',
   'Hexapoda',
   'Insecta',
   'Dicondylia',
   'Pterygota',
   'Neoptera',
   'Endopterygota',
   'Hymenoptera',
   'Tenthredinoidea',
   'Tenthredinidae',
   'Nematinae',
   'Pristiphora',
   'unclassified Pristiphora',
   'Pristiphora alpestris group sp. ZMUO 031882',
   'Pristiphora alpestris group sp. ZMUO 031882'],
  ['no rank',
   'superkingdom',
   'clade',
   'kingdom',
   'clade',
   'clade',
   'clade',
   'clade',
   'clade',
   'phylum',
   'clade',
   'clade',
   'subphylum',
   'class',
   'clade',
   'subclass',
   'infraclass',
   'cohort',
   'order',
   'superfamily',
   'family',
   'subfamily',
   'genus',
   'no rank',
   'species',
   'species']],
 [1844182,
  ['cellular organisms',
   'Bacteria',
   'Terrabacteria group',
   'Cyanobacteriot

In [34]:
print(len(taxonomyInfoRoot))


25202


In [36]:
taxonomy_path_df = pd.DataFrame(taxonomyInfoRoot, columns=['id', 'taxonomy_path', 'taxonomy_rank'])

In [37]:
taxonomy_path_df

,id,taxonomy_path,taxonomy_rank
0,2889794,"[cellular organisms, Eukaryota, Opisthokonta, ...","[no rank, superkingdom, clade, kingdom, clade,..."
1,1844182,"[cellular organisms, Bacteria, Terrabacteria g...","[no rank, superkingdom, clade, clade, phylum, ..."
2,262500,"[cellular organisms, Archaea, Euryarchaeota, S...","[no rank, superkingdom, phylum, clade, class, ..."
3,1656863,"[cellular organisms, Bacteria, Pseudomonadota,...","[no rank, superkingdom, phylum, class, order, ..."
4,2441769,"[cellular organisms, Eukaryota, Opisthokonta, ...","[no rank, superkingdom, clade, kingdom, clade,..."
...,...,...,...
25197,194973,"[cellular organisms, Bacteria, Pseudomonadota,...","[no rank, superkingdom, phylum, class, order, ..."
25198,2123410,"[cellular organisms, Eukaryota, Opisthokonta, ...","[no rank, superkingdom, clade, kingdom, clade,..."
25199,2064994,"[Viruses, Riboviria, Orthornavirae, Negarnavir...","[superkingdom, clade, kingdom, phylum, subphyl..."
25200,74612,"[cellular organisms, Eukaryota, Opisthokonta, ...","[no rank, superkingdom, clade, kingdom, subkin..."


In [66]:
taxonomy_path_df[taxonomy_path_df['id'] == 820]['taxonomy_rank'].values

array([list(['0', 'no rank', 'superkingdom', 'clade', 'clade', 'phylum', 'class', 'order', 'family', 'genus', 'species'])],
      dtype=object)

In [8]:
taxonomyRestPath = keys['taxonomyResFile']
print(taxonomyRestPath)

./data/taxonomy_path.csv


In [47]:
taxonomy_path_df.to_csv(taxonomyRestPath, index=False)